In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4MB 7.6MB/s 
     |████████████████████████████████| 92kB 13.9MB/s 
     |████████████████████████████████| 460kB 50.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.5

In [ ]:
!pip install --upgrade gensim
# 4.0.1 버전으로 업그레이드

     |████████████████████████████████| 23.9MB 127kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
!pip install glove-python-binary

     |████████████████████████████████| 952kB 29.3MB/s 


# 1) 데이터 로드

현재 이미지 800장만 학습에 사용중

In [ ]:
import os, gc, json, math, time, joblib, random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

from tqdm import tqdm
from torchvision import transforms 
from torchvision.models import resnet101
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence

from gensim.models import Word2Vec 
from konlpy.tag import Mecab, Okt, Komoran
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
class CaptionDataset(Dataset):
    """
    A PyTorch Dataset class to be used in a PyTorch DataLoader to create batches.
    """

    def __init__(self, raw_data, captions_per_image=5, mode='TRAIN'):
        super(CaptionDataset, self).__init__()
        self.mode = mode
        self.data_size = len(raw_data)
        self.captions_per_image = captions_per_image
        self.raw_images, self.raw_captions, self.raw_caplens = list(map(list, zip(*raw_data)))

        self.images = torch.FloatTensor(self.raw_images)
        self.captions = torch.LongTensor(self.raw_captions)
        self.caplens = torch.LongTensor(self.raw_caplens)

    def __getitem__(self, i):
        img = self.images[i]
        caption = self.captions[i]
        caplen = self.caplens[i]

        if self.mode == 'TRAIN':
            return img, caption, caplen
        else:
            # For validation of testing, also return all 'captions_per_image' captions to find BLEU-4 score
            img_index = i // self.captions_per_image
            all_captions = self.captions[img_index * 5:(img_index + 1) * 5]
            return img, caption, caplen, all_captions

    def __len__(self):
        return self.data_size

# 2) Tokenizer

토크나이저를 구성하는 라이브러리 `mxnet`, `gluonnlp`, `sentencepiece` 설치.  
`KoBERT` 라이브러리 설치

In [ ]:
class Mecab_Tokenizer:
    def __init__(self, model_path):
        self.tokenizer = Mecab()
        self.model = Word2Vec.load(model_path)
        self.embedding = self.model.wv
        self.unk_token = "UNK"
        self.pad_token = "PAD"

        
    def tokenize(self, sentence: str):
        return self.tokenizer.morphs(sentence)
    
    def convert_ids_to_tokens(self, ids: list):
        unk_idx = self.embedding.key_to_index[self.unk_token]
        tokens = []
        
        for idx in ids:
            try: tokens.append(self.embedding.index_to_key[idx])
            except: tokens.append(self.embedding.index_to_key[unk_idx])
        return tokens
    
    def convert_tokens_to_ids(self, tokens: list):
        unk_token = self.unk_token
        ids = []
        for token in tokens:
            try: ids.append(self.embedding.key_to_index[token])
            except: ids.append(self.embedding.key_to_index[unk_token])
        return ids
    
    def get_word_map(self):
        return self.embedding.key_to_index
    
    def get_vocab_size(self):
        return len(self.get_word_map())
    
    def get_embedding_dim(self):
        return self.model.vector_size
    
    def get_pretrained_embedding(self):
        vocab_size = self.get_vocab_size()
        embedding_size = self.get_embedding_dim()
        padding_idx = self.model.wv.key_to_index[self.pad_token]
        
        embedding = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=embedding_size, 
                                      padding_idx=padding_idx)
        embedding.weight = torch.nn.Parameter( torch.tensor(self.model.wv.vectors, dtype=torch.float) )
        return embedding

# 3) 모델링


In [ ]:
class Encoder(nn.Module):
    def __init__(self, encoded_image_size=14):
        super(Encoder, self).__init__()
        self.enc_image_size = encoded_image_size

        resnet = resnet101(pretrained=True)

        modules = list(resnet.children())[:-2]
        self.resnet = nn.Sequential(*modules)

        self.adaptive_pool = nn.AdaptiveAvgPool2d((encoded_image_size, encoded_image_size))
        self.fine_tune()

    def forward(self, images):
        """
        :param images: images, a tensor of dimensions (batch_size, 3, image_size, image_size)
        :return: encoded images
        """
        # (batch_size, 2048, image_size/32, image_size/32)
        out = self.resnet(images)
        # (batch_size, 2048, encoded_image_size, encoded_image_size)
        out = self.adaptive_pool(out)
        # (batch_size, encoded_image_size, encoded_image_size, 2048)
        out = out.permute(0, 2, 3, 1)

        # print('출력해보기_image_feature', out)
        return out

    def fine_tune(self, fine_tune=True):
        """
         Allow or prevent the computation of gradients for convolutional blocks 2 through 4 of the encoder.

         :param fine_tune: Allow?
        """
        # parameter update되지 않도록 고정
        for p in self.resnet.parameters():
            p.requires_grad = False
        # If fine-tuning, only fine-tune convolutional blocks 2 through 4
        for c in list(self.resnet.children())[5:]:
            for p in c.parameters():
                p.requires_grad = fine_tune

In [ ]:
class Attention(nn.Module):
    def __init__(self, encoder_feature_size, decoder_hidden_size, attention_size):
        super(Attention, self).__init__()
        self.encoder_feature_size = encoder_feature_size
        self.decoder_hidden_size = decoder_hidden_size
        self.attention_size = attention_size

        self.encoder_att = nn.Linear(encoder_feature_size, attention_size)
        self.decoder_att = nn.Linear(decoder_hidden_size, attention_size)
        self.f_beta = nn.Linear(decoder_hidden_size, encoder_feature_size)
        self.full_att = nn.Linear(attention_size, 1)

    def forward(self, encoder_output, decoder_hidden):
        # encoder_output : (batch_size, num_pixels, encoder_feature_size)
        # decoder_hideen: (batch_size, decoder_hidden_size)
        # (batch_size, num_pixels, attention_size)
        att1 = self.encoder_att(encoder_output)
        # (batch_size, attention_size)
        att2 = self.decoder_att(decoder_hidden)

        # att2.unsqueeze(1) -> (batch_size, 1, attention_size)
        # att1 + att2.unsqueeze(1) -> (batch_size, num_pixels, attention_size)
        # self.full_att(att1+att2.unsqueeze(1)) -> (batch_size, num_pixels, 1)
        # (batch_size, num_pixels) <- 여기가 이해되지 X
        att = self.full_att(F.relu(att1 + att2.unsqueeze(1))).squeeze(2)
        alpha = F.softmax(att, dim=1)  # (batch_size, num_pixels)

        # encoder_output : (batch_size, num_pixels, encoder_feature_size)
        # alpha.unsqueeze(2) : (batch_size, num_pixels, 1)
        # encoder_output*alpha.unsqueeze(2) -> (batch_size, num_pixels, encoder_feature_size)
        attention_weighted_encoding = (encoder_output * alpha.unsqueeze(2)).sum(dim=1)  # (batch_size, encoder_feature_size)

        gate = torch.sigmoid(self.f_beta(decoder_hidden)) # [batch_size, enc_feature_size]

        # hadamard product (gate and attention_weighted_encoding)
        attention_weighted_encoding = gate * attention_weighted_encoding # [batch_size, enc_feature_size]

        return attention_weighted_encoding

In [ ]:
class Decoder(nn.Module):
    def __init__(self, attention, tokenizer, hidden_size, dropout=0.5):
        super(Decoder, self).__init__()
        self.embedding_size = tokenizer.get_embedding_dim()
        self.output_size = tokenizer.get_vocab_size()

        self.embedding = tokenizer.get_pretrained_embedding()
        self.attention = attention
        self.lstm = nn.LSTM(self.embedding_size + self.attention.encoder_feature_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, self.output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, encoder_outputs, captions, hidden, cell):
        attention_weights = self.attention(encoder_outputs, hidden)
        
        # embedding: [batch_size, 1, embedding_size]
        # attention: [batch_size, encoder_feature_size]

        hidden = hidden.unsqueeze(0)
        cell = cell.unsqueeze(0)

        # [batch_size, 1]
        token = captions.unsqueeze(1)
        embedded = self.dropout(self.embedding(token))
        attn = attention_weights.unsqueeze(1)

        lstm_input = torch.cat([embedded, attn], dim=2)
        lstm_output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))

        output = self.fc(lstm_output)

        return output.squeeze(1), hidden.squeeze(0), cell.squeeze(0)

In [ ]:
class ImageCaptioner(nn.Module):
    def __init__(self, tokenizer, encoder_feature_size, decoder_hidden_size, attention_size, dropout=0.5):
        super(ImageCaptioner, self).__init__()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.encoder_feature_size = encoder_feature_size
        self.decoder_hidden_size = decoder_hidden_size
        self.attention_size = attention_size

        self.encoder = Encoder()
        self.attention = Attention(encoder_feature_size, decoder_hidden_size, attention_size)
        self.decoder = Decoder(self.attention, tokenizer, decoder_hidden_size)

        self.init_h = nn.Linear(self.encoder_feature_size, self.decoder_hidden_size)
        self.init_c = nn.Linear(self.encoder_feature_size, self.decoder_hidden_size)
        
    def forward(self, images, captions, caption_lengths):
        # images: 3 * 256 * 256 으로 인코딩된거 -> [batch_size, 3, 256, 256]
        # captions: 인코딩되고 패딩까지 추가된 토큰들 -> [batch_size, cap_len]
        # caption_lengths: 패딩이 추가안된 순수 문장 길이들 -> [batch_size]
        batch_size, cap_len = captions.size()

        # img -> encoder [batch_size, img_size, img_size, enc_feature_size]
        encoder_outputs = self.encoder(images).view(batch_size, -1, encoder_feature_size)
        num_pixels = encoder_outputs.size(1)

        # caption length 별 내림차순 정렬
        caption_lengths, sort_ind = caption_lengths.sort(dim=0, descending=True)
        encoder_outputs = encoder_outputs[sort_ind]
        captions = captions[sort_ind]

        hidden, cell = self.init_hidden_state(encoder_outputs)  # [batch_size, decoder_hidden_size]
        
        # Start Symbol 은 디코딩하지 않음!!!
        captions = captions[:, 1:]
        decode_lengths = (caption_lengths - 1).tolist()

        # pack_padded_sequence: <PAD> 토큰들을 죄다 지워줌!!
        # batch_sizes: <PAD> 토큰을 지우니깐 문장 길이가 달라짐 
        # -> 디코더 스텝별로 들어가는 데이터 길이가 달라지니깐 
        # -> 스텝마다 몇개가 들어가는지 리턴!!!
        captions_packed = pack_padded_sequence(captions, decode_lengths, batch_first=True)
        captions_packed_data = captions_packed.data
        batch_sizes = captions_packed.batch_sizes

        # 예측값 저장하는 tensor를 만듦
        vocab_size = tokenizer.get_vocab_size()
        predictions = torch.zeros(batch_size, cap_len, vocab_size).to(self.device)

        index = 0
        num_batch_sizes = len(batch_sizes)

        for t in range(num_batch_sizes):
            batch_size = batch_sizes[t]

            encoder_outputs = encoder_outputs[:batch_size]
            decoder_input = captions_packed_data[index:index + batch_size]
            hidden = hidden[:batch_size]
            cell = cell[:batch_size]

            decoder_output, hidden, cell = self.decoder(encoder_outputs, decoder_input, hidden, cell)
            predictions[:batch_size, t] = decoder_output

            index += batch_size

        # predictions: 예측값 (전체 단어들에 대한 확률들 싹 다) -> [batch_size, cap_len, vocab_size]
        # captions: 정렬된 정답들
        # decode_lengths: 정렬된 문장 길이들
        # sort_ind: 원본에서의 순서 [2, 4, 1, 3, 5] => [4, 1, 3, 0, 2]
        return predictions, captions, decode_lengths, sort_ind

    def init_hidden_state(self, encoder_outputs):
        mean = encoder_outputs.mean(dim=1)
        hidden = self.init_h(mean)
        cell = self.init_c(mean)

        return hidden, cell

# 4) 학습 및 검증

In [ ]:
def train(model, train_loader, criterion, optimizer, device='cpu'):
    epoch_loss = 0

    model.train()

    for _, (images, captions, caplens) in enumerate(train_loader):
        images = images.to(device)
        captions = captions.to(device)
        caplens = caplens.to(device)

        optimizer.zero_grad()

        # predictions: [batch_size, cap_len, vocab_size]
        # caption: [batch_size, cap_len]
        predictions, captions, caplens, sort_ind = model(images, captions, caplens)
        
        predictions = pack_padded_sequence(predictions, caplens, batch_first=True).data
        captions = pack_padded_sequence(captions, caplens, batch_first=True).data

        loss = criterion(predictions, captions)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

In [ ]:
def validate(model, val_loader, criterion, device='cpu'):
    epoch_loss = 0

    model.eval()

    candidates = []
    references = []

    with torch.no_grad():
        for _, (images, captions, caplens, all_captions) in enumerate(val_loader):
            images = images.to(device)
            captions = captions.to(device)
            caplens = caplens.to(device)

            predictions, captions, caplens, sort_ind = model(images, captions, caplens)
    
            preds = pack_padded_sequence(predictions, caplens, batch_first=True).data
            captions = pack_padded_sequence(captions, caplens, batch_first=True).data

            loss = criterion(preds, captions)

            epoch_loss += loss.item()

            # torch.max: 최댓값을 리턴해준다 => value, index 를 리턴해주는데
            # 우리는 index 만 필요하다. 왜? index_to_word 토큰화 이미 했으니깐
            # top1 => [batch_size, cap_len]
            _, top1 = torch.max(predictions, dim=2)
            batch_size, _ = top1.size()
            
            stop_words = ['[UNK]', '[CLS]', '[SEP]', '[MASK]', '[PAD]', 'UNK', 'CLS', 'SEP', 'MASK', 'PAD']

            for i in range(batch_size):
                # convert top1(우리가 생성한 토큰) to tokens: Candidate
                prediction_translated = [
                    token for token in tokenizer.convert_ids_to_tokens(top1[i])
                    if token not in stop_words
                ]
                #print(top1[i])

                # convert all_captions(이미지에 대한 캡션 5개) to tokens: References
                all_captions_translated = [
                    [
                        token for token in tokenizer.convert_ids_to_tokens(caption) 
                        if token not in stop_words
                    ] 
                    for caption in all_captions[i]
                ]

                candidates.append(prediction_translated)
                references.append(all_captions_translated)
                #print("[PRED]", prediction_translated)
                #print("[ANSW]", all_captions_translated)


    epoch_loss /= len(val_loader)    
    bleu_score = corpus_bleu(references, candidates, emulate_multibleu=True)

    return epoch_loss, bleu_score

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
data_dir = "/content/drive/MyDrive/졸프/3. improved model/효은_data augmentation"
model_dir = "/content/drive/MyDrive/졸프/1. data/model"
komoran_dict = os.path.join(model_dir, "userdict.txt")
mecab_w2v_model_path = os.path.join(model_dir, "MECAB_W2V.model")


device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load pretrained word embeddings
tokenizer = Mecab_Tokenizer(mecab_w2v_model_path)

# model parameters
encoder_feature_size = 2048
decoder_hidden_size = 1024
attention_size = 1024 #512
dropout = 0.5

# hyper parameters
num_epochs = 40
batch_size = 32
learning_rate = 0.001

model = ImageCaptioner(tokenizer, encoder_feature_size, decoder_hidden_size, attention_size, dropout=dropout).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# transform = transforms.Compose([
#     transforms.Normalize((0.485, 0.456, 0.406),
#                         (0.229, 0.224, 0.225))])

raw_train_data = joblib.load(os.path.join(data_dir, 'MECAB_W2V_NON_AUG_TRAIN2.pickle'))
raw_val_data = joblib.load(os.path.join(data_dir, 'MECAB_W2V_VAL2.pickle'))

train_dataset = CaptionDataset(raw_data=raw_train_data, mode='TRAIN')
val_dataset = CaptionDataset(raw_data=raw_val_data, mode='VAL')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
#gc.collect()
#torch.cuda.empty_cache()

for epoch in range(num_epochs):
    start_time = time.time()

    train_loss = train(model, train_loader, criterion, optimizer, device=device)
    val_loss, bleu_score = validate(model, val_loader, criterion, device=device)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print('Epoch: [{:02d}/{:02d}] | Time: {:02d}m {:02d}s | Train Loss: {:.4f} | Val. Loss: {:.4f} | BLEU Score: {:0.4f}'.format(
        epoch + 1, num_epochs, epoch_mins, epoch_secs, train_loss, val_loss, bleu_score))

RuntimeError: ignored

In [ ]:
# model_dir = os.path.join(BASE_DIR, 'output')

# torch.save(model.state_dict(), os.path.join(model_dir, 'savepoint_pt'))

In [ ]:
def predict(input_data, model_class, model_path, tokenizer):
    # input_data: 하나의 이미지 인풋 (imread 로 읽어들인)!
    # model_class: 모델 클래스
    # model_path: 우리가 모델을 저장한 곳!
    model_class.load_state_dict(torch.load(model_path))
    model_class.eval()

    predictions, _, _, _ = model_class(images, captions, caplens)

    _, top1 = torch.max(predictions, dim=2)

    return [
        token for token in tokenizer.convert_ids_to_tokens(top1)
        if token not in stop_words
    ]